In [9]:
!pip install torch torchvision
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
# import torchvision.transforms as transforms
from torchvision import datasets, transforms
import torch.optim as optim

The goal is to reduce parameters without drastically reducing channels, by adding a 1×1 bottleneck convolution before ConvBlock4. This reduces channel depth, so ConvBlock4 becomes much smaller while still preserving capacity and receptive field structure.

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ciferNet(nn.Module):
    def __init__(self):
        super(ciferNet, self).__init__()

        # Conv Block 1
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels= 16, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), stride=2, padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(32),
        )

        # Conv Block 2
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, 64, kernel_size=(3, 3), stride=2, padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(64),
        )

        # Conv Block 3
        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(128),
        )

        # **Bottleneck layer (1x1 convolution to reduce channels)**
        self.bottleneck = nn.Conv2d(in_channels=128, out_channels=96, kernel_size=(1, 1), bias=False)  # Reduce channels before ConvBlock4

        # Conv Block 4
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=96, out_channels=128, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(128),
        )

        # GAP and classifier
        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Conv2d(in_channels=128, out_channels=10, kernel_size=(1, 1), bias=False)

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.bottleneck(x)      # Reduce channels
        x = self.convblock4(x)
        x = self.gap(x)
        x = self.classifier(x)
        x = x.view(x.size(0), -1)
        return F.log_softmax(x, dim=-1)


In [11]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = ciferNet().to(device)
summary(model, input_size=(3, 32, 32))

cpu
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
              ReLU-2           [-1, 16, 32, 32]               0
       BatchNorm2d-3           [-1, 16, 32, 32]              32
            Conv2d-4           [-1, 32, 16, 16]           4,608
              ReLU-5           [-1, 32, 16, 16]               0
       BatchNorm2d-6           [-1, 32, 16, 16]              64
            Conv2d-7           [-1, 64, 16, 16]          18,432
              ReLU-8           [-1, 64, 16, 16]               0
       BatchNorm2d-9           [-1, 64, 16, 16]             128
           Conv2d-10             [-1, 64, 8, 8]          36,864
             ReLU-11             [-1, 64, 8, 8]               0
      BatchNorm2d-12             [-1, 64, 8, 8]             128
           Conv2d-13            [-1, 128, 8, 8]          73,728
             ReLU-14            [-1

In [12]:
# Train Phase transformations
train_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.4741, 0.4727, 0.4733),(0.2521, 0.2520, 0.2506),)
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.4741, 0.4727, 0.4733),(0.2521, 0.2520, 0.2506),)
                                       ])

In [13]:
train = datasets.CIFAR10('./data', train=True, download=True, transform=train_transforms)
test = datasets.CIFAR10('./data', train=False, download=True, transform=test_transforms)

In [14]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
train_dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)
test_dataloader_args = dict(shuffle=False, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=False, batch_size=64)


# train dataloader
train_loader = torch.utils.data.DataLoader(train, **train_dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **test_dataloader_args)

CUDA Available? False


In [15]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes.
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss.item()) # Added .item()

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm

    pred = y_pred.float().argmax(dim=1, keepdim=True)  # get the index of the max log-probability, convert to float
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.float().argmax(dim=1, keepdim=True)  # get the index of the max log-probability, convert to float
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss) # Added .item()

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))

In [16]:
from torch.optim.lr_scheduler import StepLR

model =  ciferNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# scheduler = StepLR(optimizer, step_size=6, gamma=0.1)


EPOCHS = 8
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    # scheduler.step()
    test(model, device, test_loader)

EPOCH: 0


Loss=1.539613962173462 Batch_id=781 Accuracy=49.11: 100%|██████████| 782/782 [03:31<00:00,  3.70it/s]



Test set: Average loss: 1.1530, Accuracy: 5875/10000 (58.75%)

EPOCH: 1


Loss=1.0633013248443604 Batch_id=781 Accuracy=64.79: 100%|██████████| 782/782 [03:16<00:00,  3.98it/s]



Test set: Average loss: 0.9198, Accuracy: 6781/10000 (67.81%)

EPOCH: 2


...


Test set: Average loss: 0.6446, Accuracy: 7767/10000 (77.67%)

EPOCH: 6


Loss=0.34837964177131653 Batch_id=781 Accuracy=84.33: 100%|██████████| 782/782 [03:16<00:00,  3.98it/s]



Test set: Average loss: 0.6592, Accuracy: 7742/10000 (77.42%)

EPOCH: 7


Loss=0.6530765891075134 Batch_id=781 Accuracy=86.48: 100%|██████████| 782/782 [03:15<00:00,  3.99it/s]



Test set: Average loss: 0.6467, Accuracy: 7838/10000 (78.38%)

